In [18]:
### document data structure

from langchain_core.documents import Document

In [19]:
doc = Document (
    page_content ="This is the main text content I am using to create RAG",
    metadata= {
        "source": "example.txt",
        "pages":1,
        "author":"Pradnya Karwade" ,
        "date_created":"2025-01-01"
    }
)
doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'author': 'Pradnya Karwade', 'date_created': '2025-01-01'}, page_content='This is the main text content I am using to create RAG')

In [20]:
## create a simple txt file

import os
os.makedirs("../data/text_files",exist_ok=True)

In [21]:
sample_texts={
    "../data/text_files/python_intro.txt": """Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability. 
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular 
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility}
-Strong community support

Python is widely used in web development, data science, artificail intelligence, and automation.""",

"../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve 
from experience without being explicitly programmed. It focuses on developing computer programs 
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems

"""
}

for filepath,content in sample_texts.items():
     with open (filepath, 'w', encoding = "utf-8") as f:
           f.write (content)

print ("Done!! Sample text files are created!")

Done!! Sample text files are created!


In [22]:
###Text Loader
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader

loader = TextLoader ("../data/text_files/python_intro.txt", encoding="utf-8")
document = loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability. \nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular \nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility}\n-Strong community support\n\nPython is widely used in web development, data science, artificail intelligence, and automation.')]


In [23]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader
## load all the text files from the directory

dir_loader = DirectoryLoader (
    "../data/text_files",
    glob="**/*.txt", ##Pattern to match the files
    loader_cls = TextLoader, ##loader class to use
    loader_kwargs ={'encoding' : 'utf-8'},
    
)

documents=dir_loader.load()
documents

[Document(metadata={'source': '..\\data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve \nfrom experience without being explicitly programmed. It focuses on developing computer programs \nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n'),
 Document(metadata={'source': '..\\data\\text_files\\python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability. \nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular

In [24]:

from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
## load all the text files from the directory

dir_loader = DirectoryLoader (
    "../data/pdf",
    glob="**/*.pdf", ##Pattern to match the files
    loader_cls = PyMuPDFLoader, ##loader class to use
        
)
pdf_documents= dir_loader.load()
pdf_documents

[Document(metadata={'producer': 'Mac OS X 10.8.2 Quartz PDFContext', 'creator': 'PDFCreator Version 1.2.3', 'creationdate': "D:20130115124904Z00'00'", 'source': '..\\data\\pdf\\Attention.pdf', 'file_path': '..\\data\\pdf\\Attention.pdf', 'total_pages': 6, 'format': 'PDF 1.3', 'title': 'Helgason-2012-CR', 'author': 'Helgi', 'subject': '', 'keywords': '', 'moddate': "D:20130115124904Z00'00'", 'trapped': '', 'modDate': "D:20130115124904Z00'00'", 'creationDate': "D:20130115124904Z00'00'", 'page': 0}, page_content='Attention capabilities for AI systems\n \nHelgi Páll Helgason1, Kristinn R. Thórisson1,2 \n1Center for Analysis & Design of Intelligent Agents / School of Computer Science, Venus 2nd floor, Reykjavik \nUniversity,Menntavegur 1, 101 Reykjavik, Iceland \n2Icelandic Institute for Intelligent Machines, 2. h. Uranus, Menntavegur 1, 101 Reykjavik, Iceland \nhelgih09@ru.is, thorisson@ru.is \nKeywords: \nArtificial intelligence, attention, resource management \nAbstract: \nMuch of presen

In [25]:
#CHUNKING
from pathlib import Path
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents


def split_documents(documents, chunk_size=500, chunk_overlap=50):
    """Split documents into smaller chunks
    
    Args:
        documents: List of Document objects or raw strings.
        chunk_size: Max characters per chunk.
        chunk_overlap: Overlap between chunks.
    
    Returns:
        List of Document chunks
    """
    # Ensure all inputs are Document objects
    if isinstance(documents[0], str):
        documents = [Document(page_content=doc, metadata={}) for doc in documents]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=['\n\n', '\n', ' ', '']
    )
    
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk")
        print(f"Content : {split_docs[0].page_content[:200]}...")
        print(f"Metadata : {split_docs[0].metadata}")
    
    return split_docs


# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

# Split them into chunks
chunks = split_documents(all_pdf_documents)
print(f"Total chunks created: {len(chunks)}")

Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)


Found 4 PDF files to process

Processing: Attention.pdf
  ✓ Loaded 6 pages

Processing: Embeddings.pdf
  ✓ Loaded 5 pages

Processing: Embeddings2.pdf
  ✓ Loaded 27 pages

Processing: ObjectDetection.pdf
  ✓ Loaded 14 pages

Total documents loaded: 52
Split 52 documents into 327 chunks

Example chunk
Content : Attention capabilities for AI systems 
 
Helgi Páll Helgason1, Kristinn R. Thórisson1,2 1Center for Analysis & Design of Intelligent Agents / School of Computer Science, Venus 2nd floor, Reykjavik 
Un...
Metadata : {'producer': 'Mac OS X 10.8.2 Quartz PDFContext', 'creator': 'PDFCreator Version 1.2.3', 'creationdate': "D:20130115124904Z00'00'", 'title': 'Helgason-2012-CR', 'author': 'Helgi', 'moddate': "D:20130115124904Z00'00'", 'source': '..\\data\\pdf\\Attention.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1', 'source_file': 'Attention.pdf', 'file_type': 'pdf'}
Total chunks created: 327


In [26]:
#Embedding And VectorDB
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict,Any, Tuple 
from sklearn.metrics.pairwise import cosine_similarity



In [27]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
      """Load the SentenceTransformer model"""
      try:
        print(f"Loading embedding model: {self.model_name}")
        self.model = SentenceTransformer(self.model_name)
        print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
      except Exception as e:
        print(f"Error loading model {self.model_name}: {e}")
        raise
      

    
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args:
            texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
           raise ValueError("Model not loaded")

        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    

##Initialize the Embedding Manager

embedding_manager= EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [28]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                 name=self.collection_name,
                 metadata={"description": "PDF document embeddings for RAG"}
            )

            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
           raise ValueError("Number of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vector store...")

        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
             
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embeddings_list.append(embedding.tolist())

        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 168


In [29]:
chunks

[Document(metadata={'producer': 'Mac OS X 10.8.2 Quartz PDFContext', 'creator': 'PDFCreator Version 1.2.3', 'creationdate': "D:20130115124904Z00'00'", 'title': 'Helgason-2012-CR', 'author': 'Helgi', 'moddate': "D:20130115124904Z00'00'", 'source': '..\\data\\pdf\\Attention.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1', 'source_file': 'Attention.pdf', 'file_type': 'pdf'}, page_content='Attention capabilities for AI systems \n \nHelgi Páll Helgason1, Kristinn R. Thórisson1,2 1Center for Analysis & Design of Intelligent Agents / School of Computer Science, Venus 2nd floor, Reykjavik \nUniversity,Menntavegur 1, 101 Reykjavik, Iceland \n2Icelandic Institute for Intelligent Machines, 2. h. Uranus, Menntavegur 1, 101 Reykjavik, Iceland \nhelgih09@ru.is, thorisson@ru.is  \nKeywords: Artificial intelligence, attention, resou rce management'),
 Document(metadata={'producer': 'Mac OS X 10.8.2 Quartz PDFContext', 'creator': 'PDFCreator Version 1.2.3', 'creationdate': "D:20130115124904Z00'00'

In [30]:
##Convert Text to Embeddings

texts=[doc.page_content for doc in chunks]
texts

['Attention capabilities for AI systems \n \nHelgi Páll Helgason1, Kristinn R. Thórisson1,2 1Center for Analysis & Design of Intelligent Agents / School of Computer Science, Venus 2nd floor, Reykjavik \nUniversity,Menntavegur 1, 101 Reykjavik, Iceland \n2Icelandic Institute for Intelligent Machines, 2. h. Uranus, Menntavegur 1, 101 Reykjavik, Iceland \nhelgih09@ru.is, thorisson@ru.is  \nKeywords: Artificial intelligence, attention, resou rce management',
 'Abstract: Much of present AI research is based on t he assumption of computational systems with infinit e resources, \nan assumption that is either explicitly stated or i mplicit in the work as researchers ignore the fact that most \nreal-world tasks must be finished within certain ti me limits, and it is the role of intelligence to ef fectively \ndeal with such limitations. Expecting AI systems to  g i v e  e q u a l  t r e a t m e n t  t o  e v e r y  p i e c e  o f  d a t a  t h e y',
 'encounter is not appropriate in most real-wo

In [31]:
##Generate Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##Store int he vector database
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 327 texts...


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches: 100%|██████████| 11/11 [00:11<00:00,  1.02s/it]


Generated embeddings with shape: (327, 384)
Adding 327 documents to vector store...
Successfully added 327 documents to vector store
Total documents in collection: 495


In [32]:
# Retriever Pipeline From VectorStore
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever

        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query

        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold

        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            # Process results
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance

                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No Documents found")

            return retrieved_docs

        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []
        

rag_retriever=RAGRetriever(vectorstore,embedding_manager)


In [33]:
rag_retriever

In [34]:
rag_retriever.retrieve("What is attention is all you need")

Retrieving documents for query: 'What is attention is all you need'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 46.54it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_467e271a_8',
  'content': 'has provided us with a prime example in human \nattention; a cognitive function that enables us to \nfocus our limited resources selectively on \ninformation that is most important to us at any giv en \nmoment as we perform various tasks while',
  'metadata': {'moddate': "D:20130115124904Z00'00'",
   'title': 'Helgason-2012-CR',
   'producer': 'Mac OS X 10.8.2 Quartz PDFContext',
   'creator': 'PDFCreator Version 1.2.3',
   'source_file': 'Attention.pdf',
   'file_type': 'pdf',
   'page_label': '1',
   'page': 0,
   'author': 'Helgi',
   'creationdate': "D:20130115124904Z00'00'",
   'doc_index': 8,
   'source': '..\\data\\pdf\\Attention.pdf',
   'total_pages': 6,
   'content_length': 239},
  'similarity_score': 0.11798709630966187,
  'distance': 0.8820129036903381,
  'rank': 1},
 {'id': 'doc_bc71b03b_8',
  'content': 'has provided us with a prime example in human \nattention; a cognitive function that enables us to \nfocus our limited resources s

In [46]:
rag_retriever.retrieve("Training Optimization")

Retrieving documents for query: 'Training Optimization'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 48.25it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


[{'id': 'doc_954cf6ec_172',
  'content': 'all training data, set the learning rate to 2e-5, and train for 8k ste ps, keeping all other\nconﬁgurations the same as in the ﬁrst stage. We employ a batch size of 256 for all data\nusing the InfoNCE loss (i.e., retrieval and classiﬁcation), considerin g data using the\ncosent loss (i.e., NLI), due to lower memory consumption from the ab sence of forward\ncomputation for negative samples, the batch size is set to 768. Acr oss all stages, we',
  'metadata': {'arxivid': 'https://arxiv.org/abs/2508.21632v1',
   'creator': 'arXiv GenPDF (tex2pdf:)',
   'content_length': 441,
   'keywords': '',
   'total_pages': 27,
   'page_label': '14',
   'page': 13,
   'source_file': 'Embeddings2.pdf',
   'doi': 'https://doi.org/10.48550/arXiv.2508.21632',
   'license': 'http://creativecommons.org/licenses/by/4.0/',
   'author': 'Peng Yu; En Xu; Bin Chen; Haibiao Chen; Yinfei Xu',
   'creationdate': '2025-09-01T00:50:53+00:00',
   'moddate': '2025-09-01T00:50:5